# importing all the python modules

In [ ]:
# general
import os
import glob
import shutil


#dates
from datetime import date
from datetime import time
from datetime import datetime

# data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sb

# biopython
from Bio import SeqIO, AlignIO
from Bio.SeqRecord import SeqRecord

# phylogenetics
from ete3 import Tree, TreeStyle, NodeStyle, AttrFace, faces, CircleFace
from ete3 import SeqMotifFace, add_face_to_node, TextFace, CircleFace
from ete3 import Nexml

# set display options
pd.options.display.max_columns = 50
os.environ['QT_QPA_PLATFORM']='offscreen'

In [ ]:
# definitions
bi_covid_dataset = 'path to file'
concatenated_assembly_metrics = 'path to file'

# wd = '~/buena_vista_study/'
wd = 'path to file'
os.chdir(wd)
os.getcwd()

In [ ]:
# import the post_assembled_genomes.py script
os.chdir('path to file')
import indel_finder as indel_finder

# import the nextclade parser function
import nextclade_json_parser as parser

# import clean horizon query script
# import clean_horizon_cov_pui_query as clean_query
import clean_horizon_covpui_varseq_fix_collection_date_2 as clean_query 

os.chdir(wd)

# 1. merging pui and varseq

In [ ]:
# 1 = merging  bicovid and varseq
bicovid_path = 'path to file'
varseq_path = 'path to file'

bicovid_varseq = clean_query.main(bicovid_path =  bicovid_path, varseq_path = varseq_path)
bicovid_varseq = bicovid_varseq.reset_index(drop = True)
bicovid_varseq.head()

# save files
outfile = os.path.join(wd, '01_bicovid_varseq.csv')
bicovid_varseq.to_csv(outfile, index = False)

# 2. fix collection dates from pui and cv_var

In [ ]:
# to fix collection dates from  bi_covid and cv_var
path = os.path.join(wd, '01_bicovid_varseq.csv')
query = pd.read_csv(path, dtype = {"accession_id" : object})

# drop test clients
crit = query.cust_name != 'Test Client- For Testing only'
query = query[crit]
query = query.reset_index(drop = True)

# replace missing collection_dates with receive dates
crit = query.collection_date.isna()
print('number of NAs collection dates: %d' % query[crit].shape[0])
query.collection_date = query.collection_date.fillna('')
print(crit.shape)

# how many collection dates need to be corrected
crit = query.collection_date < '2020-03-01'
print('will we replace %d collection dates (before 2020-03-01) with receive dates' % query[crit].shape[0])
print('')
print(crit.shape)



collection_date_mod_list = []

for row in range(query.shape[0]):
    
    collection_date = query.collection_date[row]
    receive_date = query.receive_date[row]
#     print(collection_date, receive_date, query.cust_name[row], query.first_name[row])
    
    if collection_date < '2020-03-01' or collection_date == '':
        collection_date_mod_list.append(receive_date)
    else:
        collection_date_mod_list.append(collection_date)
        
query['collection_date_mod'] = collection_date_mod_list
query.head()


outfile = os.path.join(wd, '02_collection_date_mod.csv')
query.to_csv(outfile, index = False)

In [ ]:
path = os.path.join (wd, "02_collection_date_mod.csv")
df = pd.read_csv (path, dtype = {'accession_id' : object})

def addprefix (accession_id):
    return 'CO-CDPHE-%s' % accession_id

fasta_header = df.apply(lambda x: addprefix (x.accession_id), axis =1)
df.insert(value = fasta_header, column = 'fasta_header', loc =0)
df.dtypes

# make identifier
# to get the number of individuals
def make_identifier(last_name, first_name, dob):
    return "%s_%s_%s" % (last_name, first_name, dob)

df.first_name = df.first_name.str.capitalize()
df.first_name = df.first_name.str.strip()
df.last_name = df.last_name.str.capitalize()
df.last_name = df.last_name.str.strip()
df.dob= pd.to_datetime(df.dob).dt.date

# df.last_name= df.last_name.str.capitalize()



identifier = df.apply(lambda x:make_identifier(x.last_name, x.first_name, x.dob), axis = 1)
df.insert(value = identifier, column = "person_id", loc = 0)

print(df.shape)

# save
outfile = os.path.join(wd, '02_collection_date_mod.csv')
df.to_csv(outfile, index = False)
df


In [ ]:
# adding the "blank" staff members into the dataset
path = os.path.join (wd, 'path to file')
filename = pd.read_csv (path, dtype = {'accession_id' : object})


filename.head()
filename.collection_date = pd.to_datetime(filename.collection_date, format = '%m/%d/%Y')
filename = filename.rename(columns = {'collection_date' : 'collection_date_mod'})
filename = filename.accession_id.unique().tolist()

print(len(df.accession_id.unique().tolist())) # number of indivduals taht have tested positive 


In [ ]:
# filtering dataset to filename
path = os.path.join (wd, "02_collection_date_mod.csv")
df = pd.read_csv (path, dtype = {'accession_id' : object})

crit = df.cust_name == 'filter file'
crit2 = df.accession_id.isin(filename)
filename_filtered = df[crit | crit2]
print(filename_filtered.shape)

# save
outfile = os.path.join(wd, '03_filename_filtered.csv')
filename_filtered.to_csv(outfile, index = False)
filename_filtered

In [ ]:
# dropping duplicates in filename filtered samples
path = os.path.join (wd, "03_filename_filtered.csv")
df = pd.read_csv (path, dtype = {'accession_id' : object})

filename_dropdups = df.drop_duplicates(subset = 'person_id', keep = 'first')
print(filename_dropdups.shape)

# save
outfile = os.path.join(wd, '04_filename_dropped_duplicates.csv')
filename_dropdups.to_csv(outfile, index = False)
filename_dropdups

In [ ]:
# ensuring that person_id is only mentioned once
path = os.path.join (wd, "04_filename_dropped_duplicates.csv")
filename_person_id = pd.read_csv (path, dtype = {'accession_id' : object})


# making sure that person_id are only mentioned once
filename_person_id = filename_person_id.reset_index(drop=True)
for row in range (filename_person_id.shape[0]) : 
    person_id = filename_person_id.person_id[row]
    if person_id in corrections_dictionary.keys() : 
        print(person_id)
        filename_person_id.at[row, 'person_id'] = corrections_dictionary [person_id]
print(filename_person_id.shape)

filename_person_id = filename_person_id.drop_duplicates(subset = 'person_id', keep = 'first')
print(filename_person_id.shape)
        
# save
outfile = os.path.join(wd, '04_filename_person_id.csv')
filename_person_id.to_csv(outfile, index = False)
filename_person_id

In [ ]:
# open up the file
path = '/home/diana_ir/Documents/nextstrain/covid_sequencing/concat_metrics/concatenated_assembly_metrics_2022-04-14.csv'
seq_metrics = pd.read_csv (path, dtype = {'accession_id' : object})

drop_col = ['fasta_header']

seq_metrics = seq_metrics.drop(columns= drop_col)

path = os.path.join (wd, '04_filename_person_id.csv')
filename_person_id = pd.read_csv (path, dtype = {'accession_id' : object})

# merging pui & varseq with seq data
seq_metrics = seq_metrics.sort_values(by = 'percent_non_ambigous_bases', ascending = False)
seq_metrics = seq_metrics.drop_duplicates(subset = 'accession_id', keep = 'first')
seq_metrics = seq_metrics.set_index('accession_id')
filename_person_id = filename_person_id.set_index('accession_id')

# joining concatenated assembly metrics with filtered data
j = filename_person_id.join(seq_metrics, how = 'left')
j = j.reset_index() #don't put drop because we don't want to drop accession id
print(j.shape)

# save
outfile = os.path.join(wd, '05_seq_metrics_filename_person_id.csv')
j.to_csv(outfile, index = False)

In [ ]:
# open up the file
path = os.path.join (wd, '05_seq_metrics_filename_person_id.csv')
seq_metrics_filename_person_id = pd.read_csv (path, dtype = {'accession_id' : object})

# drop NA in percent non ambiguous bases, unique to individuals
na = seq_metrics_filename_person_id.dropna(axis=0, subset = ['percent_non_ambigous_bases'])
print(na.shape)

# drop duplicates
na = na.drop_duplicates(subset = 'person_id', keep = 'first')
print(na.shape)
na

# save
outfile = os.path.join(wd, '06_droppedna_percent_non_ambiguous_bases.csv')
na.to_csv(outfile, index = False)

In [ ]:
# read in files
path = os.path.join (wd, '06_droppedna_percent_non_ambiguous_bases.csv')
na = pd.read_csv (path, dtype = {'accession_id' : object})

# has >90% coverage
na = na.sort_values(by = 'percent_non_ambigous_bases', ascending = False)
print(na.shape)

crit = na.percent_non_ambigous_bases >= 90
cov = na[crit]
print(cov.shape)

crit2 = na.percent_non_ambigous_bases <=90
crit3 = na.percent_non_ambigous_bases >1
nocov = na[crit2 & crit3]
print(nocov.shape)

crit4 = na.percent_non_ambigous_bases == 0
zero = na[crit4]
print(zero.shape)

# save
outfile = os.path.join(wd, '07_high_coverage_samples.csv')
cov.to_csv(outfile, index = False)

outfile = os.path.join(wd, '08_low_coverage_samples.csv')
nocov.to_csv(outfile, index = False)

outfile = os.path.join(wd, '08_zero_coverage_samples.csv')
zero.to_csv(outfile, index = False)

In [ ]:
# calls lineages >90% high coverage samples, remove duplications
path = os.path.join (wd, '07_high_coverage_samples.csv')
high = pd.read_csv (path, dtype = {'accession_id' : object})

# drop duplicates
high = high.sort_values(by = 'collection_date')
high = high.drop_duplicates(subset = 'person_id', keep = 'first')
print(high.shape)
high

# save
outfile = os.path.join(wd, '09_high_cov_drop_dups.csv')
high.to_csv(outfile, index = False)

In [ ]:
# acquire leftover samples that didn't link to TriCounty Epi data
path = os.path.join (wd, '08_low_coverage_samples.csv')
low = pd.read_csv (path, dtype = {'accession_id' : object})

# drop duplicates
low = low.sort_values(by = 'collection_date')
low = low.drop_duplicates(subset = 'person_id', keep = 'first')
print(low.shape)
low

# save
outfile = os.path.join(wd, '10_low_cov_drop_dups.csv')
low.to_csv(outfile, index = False)

# high coverage --> assembly metrics and fasta

In [ ]:
# open data up
path = os.path.join (wd, '09_high_cov_drop_dups.csv')
high = pd.read_csv (path, dtype = {'accession_id' : object})

# merging with lineage
outfile = '09_highcov_assembly_metrics.csv'
high.to_csv(outfile, index = False)
high

# using filtered assembly metrics we want to pull to out the fasta files for those sequences
out_fasta = '09_highcov_sequences.fasta'
if os.path.isfile(out_fasta):
    os.remove(out_fasta)

n=0
for row in range(high.shape[0]):
    n=n+1
    accession_id = high.accession_id[row]
    seq_run = high.seq_run[row]
    
    fasta_file_path = '/home/diana_ir/Documents/nextstrain/covid_sequencing/fasta_files/%s/%s_consensus_renamed.fa' % (seq_run, accession_id)
    
    print(n, 'adding %s_consensus_renamed.fa from %s to concatenated fasta file' % (accession_id, seq_run))
    record = SeqIO.read(fasta_file_path, 'fasta')
    with open(out_fasta, 'a') as out_handle:
        SeqIO.write(record, out_handle, 'fasta')
        
# on terminal, run pangolin on these files


In [ ]:
# read file in
path = os.path.join (wd, '09_highcov_assembly_metrics.csv')
high = pd.read_csv(path, dtype = {'accession_id' : object})
print(high.shape)

# join lineage report with assembly metrics
path = os.path.join (wd, '09_highcov_lineage_report.csv')
pangolin = pd.read_csv(path, dtype = {'taxon': object})
print(pangolin.shape)

drop_col = ['pangolin_version']
high = high.drop(columns=drop_col)

# merge file together
highcov_metrics_lineage = high.join(pangolin, how = 'left')
highcov_metrics_lineage = highcov_metrics_lineage.reset_index()
print(highcov_metrics_lineage.shape)

# save file
outfile = '11_highcov_metrics_pangolin.csv'
highcov_metrics_lineage.to_csv(outfile, index= False)
highcov_metrics_lineage.shape

# low coverage --> assembly metrics and fasta

In [ ]:
# open data up
path = os.path.join (wd, '10_low_cov_drop_dups.csv')
low = pd.read_csv (path, dtype = {'accession_id' : object})

low['lineage'] = 'low quality sequences'

# save file
outfile = '12_lowcov_metrics_lqseq.csv'
low.to_csv(outfile, index= False)
low.shape


# combining high and low coverage metrics pangolin data

In [ ]:
# read files
path = os.path.join (wd, '12_lowcov_metrics_lqseq.csv')
low = pd.read_csv(path, dtype = {'accession_id' : object})
print(low.shape)

path = os.path.join (wd, '11_highcov_metrics_pangolin.csv')
high = pd.read_csv(path, dtype = {'accession_id' : object})
print(high.shape)


# concatenate files together
comb = pd.concat([high, low])
comb.reset_index()
print(comb.shape)

# save file
outfile = '13_combined_metrics_pangolin.csv'
comb.to_csv(outfile, index= False)

# read in 'no sequences' dataset

In [ ]:
# read files
path = os.path.join (wd, '05_seq_metrics_filename_person_id.csv')
na = pd.read_csv(path, dtype = {'person_id' : object})
na = na.sort_values (by = 'person_id')
print(na.shape)

# creating a crit that drops high / low coverage
crit = na['percent_non_ambigous_bases'].isnull()
noseq = na[crit]
print(noseq.shape)

# drop duplicates
noseq = noseq.sort_values(by = 'collection_date')
noseq = noseq.drop_duplicates(subset = 'person_id', keep = 'first')
print(noseq.shape)

# save file
outfile = '14_no_sequences.csv'
noseq.to_csv(outfile, index= False)

In [ ]:
# read files
path = os.path.join (wd, '14_no_sequences.csv')
noseq = pd.read_csv(path, dtype = {'person_id' : object})
noseq = noseq.sort_values (by = 'person_id')
print(noseq.shape)

noseq['lineage'] = 'not sequenced'

# save file
outfile = '15_no_sequences_labeled.csv'
noseq.to_csv(outfile, index= False)

# combining high/low seq with noseq datasets

In [ ]:
# read files
path = os.path.join (wd, '13_combined_metrics_pangolin.csv')
comb = pd.read_csv(path, dtype = {'accession_id' : object})
print(comb.shape)

path = os.path.join (wd, '15_no_sequences_labeled.csv')
noseq = pd.read_csv(path, dtype = {'accession_id' : object})
print(noseq.shape)

# concatenate files together
comb_noseq = pd.concat([comb, noseq])
comb_noseq.reset_index()
print(comb_noseq.shape)

# save file
outfile = '16_final_version.csv'
comb_noseq.to_csv(outfile, index= False)

In [ ]:
# read files
path = os.path.join (wd, '16_final_version.csv')
final = pd.read_csv(path, sep = '\t', dtype = {'accession_id' : object})
print(final.shape)

# aggreate by date
final.collection_date_mod = pd.to_datetime(final['collection_date_mod']) - pd.to_timedelta(7, unit= 'd')
final = final.sort_values(by = 'collection_date_mod')


weekly_counts = final.groupby(['lineage', pd.Grouper(key ='collection_date_mod', freq = 'W-SUN')]).size().unstack('lineage')
weekly_counts = weekly_counts.reset_index()
weekly_counts = weekly_counts.sort_values(by = 'collection_date_mod')
weekly_counts = weekly_counts.reset_index(drop = True)


weekly_counts.collection_date_mod = weekly_counts.collection_date_mod.dt.strftime('%d-%b-%Y')

weekly_counts = weekly_counts.fillna(0)

total_list = []
for row in range(weekly_counts.shape[0]):
    week_total = 0
    for column in weekly_counts.columns:
        if column != 'collection_date_mod':
            week_total = int(week_total + weekly_counts.at[row, column])
    total_list.append(week_total)
weekly_counts['total'] = total_list
weekly_counts

# save file
outfile = 'TABLE_weekly_counts.csv'
weekly_counts.to_csv(outfile, index= False)

In [ ]:
# read files
path = os.path.join (wd, 'TABLE_weekly_counts.csv')
weekly_counts = pd.read_csv(path, dtype = {'accession_id' : object})
print(weekly_counts.shape)

weekly_counts.columns

In [ ]:
lineages = ['AY.100', 'AY.103', 'AY.122', 'AY.25', 'AY.26',
       'AY.3', 'AY.3.1', 'AY.37', 'AY.39', 'AY.4', 'AY.44', 'AY.54', 'B.1',
       'B.1.1.222', 'B.1.1.519', 'B.1.1.7', 'B.1.2', 'B.1.229', 'B.1.234',
       'B.1.240', 'B.1.243', 'B.1.396', 'B.1.400', 'B.1.403', 'B.1.429',
       'B.1.526', 'B.1.570', 'B.1.595', 'B.1.617.2', 'low quality sequences',
       'not sequenced']
lineages.sort(reverse = True)
lineages

In [ ]:
lineages = ['not sequenced',
 'low quality sequences',
 'B.1.617.2',
 'B.1.595',
 'B.1.570',
 'B.1.526',
 'B.1.429',
 'B.1.403',
 'B.1.400',
 'B.1.396',
 'B.1.243',
 'B.1.240',
 'B.1.234',
 'B.1.229',
 'B.1.2',
 'B.1.1.7',
 'B.1.1.519',
 'B.1.1.222',
 'B.1',
 'AY.54',
 'AY.44',
 'AY.4',
 'AY.39',
 'AY.37',
 'AY.3.1',
 'AY.3',
 'AY.26',
 'AY.25',
 'AY.122',
 'AY.103',
 'AY.100']

colors = [ 'orange', #1
          'green', #2
          'gold', #3
         'olivedrab', #4
         'lightseagreen',#5
         'deepskyblue', #6
         'navy', #7
         'blueviolet', #8
         'plum', #9
         'purple', #10
         'deeppink', #11
          'yellow', #12
        'red', #13
         'pink', #14
         'blue', #15
         'darkorange', #16
         'lightblue',#17
         'orange', #18
          'green', #19
          'gold', #20
         'olivedrab', #21
         'lightseagreen',#22
         'deepskyblue', #23
         'navy', #24
         'blueviolet', #25
         'plum', #26
         'purple', #27
         'deeppink', #28
          'yellow', #29
           'red', #30
         'pink', #31
         'blue', #32
         'darkorange', #33
         'lightblue',#34
          'gold' #35
         ] 
color_dict = dict(zip(lineages, colors))
color_dict

In [ ]:
WC = weekly_counts.collection_date_mod.tolist()
fig, ax = plt.subplots(figsize=(90,30))
X =range(weekly_counts.shape[0])


for i in range(len(lineages)):
    print(i, lineages[i])
    if i == 0:
        #create the bar
        plt.bar(x = X, 
                height = weekly_counts[lineages[i]], 
                color = color_dict[lineages[i]], 
                edgecolor = 'white', 
                width = 0.8 )
    
    else:
        bottom_values = 0
        for k in range(i):
            if k < i :
                bottom_values = bottom_values + weekly_counts[lineages[k]]
        #create the bar
        plt.bar(x = X, 
                height = weekly_counts[lineages[i]], 
                bottom = bottom_values, 
                color = color_dict[lineages[i]], 
                edgecolor = 'white', 
                width = 0.8 )
    
    
# plt.bar(x=X, height = weekly_counts.total, width = 0.8)
plt.style.use('seaborn-white')
plt.title('')
# plt.xlabel('date', fontsize = 56)
plt.ylabel('number of individuals tested positive', fontsize = 80)
plt.ylim((0, 200))

plt.yticks(fontsize = 70)
plt.xticks(X, WC, rotation = 90, fontsize = 60)

plt.legend(color_dict.keys(), fontsize = 70, title = 'lineage', ncol=8)
# plt.legend(ncol=3)

# plt.tight_layout()
path = os.path.join(wd, 'FIGURE_epi_curve_bv_with_lineages.jpeg')
plt.savefig(path)
plt.show